In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd

# Get the current datetime64[us, US/EST]
current_date = pd.Timestamp.now(tz='US/Eastern')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_convert("US/Eastern")

from src.data_utils import transform_ts_data_info_features_and_target
features, _ = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-02 20:54:07,369 INFO: Initializing external client
2025-03-02 20:54:07,369 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:54:08,380 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Fetching data from 2025-02-01 20:54:07.369627-05:00 to 2025-03-02 19:54:07.369627-05:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.66s) 


In [4]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-02 20:54:27,846 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 20:54:27,860 INFO: Initializing external client
2025-03-02 20:54:27,861 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:54:28,731 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [5]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [6]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,217,0.0,2025-03-02 21:00:00-05:00
1,65,0.0,2025-03-02 21:00:00-05:00
2,261,2.0,2025-03-02 21:00:00-05:00
3,69,0.0,2025-03-02 21:00:00-05:00
4,155,0.0,2025-03-02 21:00:00-05:00
...,...,...,...
251,168,0.0,2025-03-02 21:00:00-05:00
252,74,2.0,2025-03-02 21:00:00-05:00
253,80,0.0,2025-03-02 21:00:00-05:00
254,79,141.0,2025-03-02 21:00:00-05:00


In [7]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-02 20:54:44,900 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 20:54:44,905 INFO: Initializing external client
2025-03-02 20:54:44,906 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:54:45,750 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [8]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1212635/fs/1200268/fg/1401818


Uploading Dataframe: 100.00% |██████████| Rows 256/256 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)